In [ ]:
# Load basic libraries
import pandas as pd
import numpy as np
from functools import partial
import re

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
data = pd.read_excel("/kaggle/input/data-train/Data_Train.xlsx")
data

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop(["Price"],axis = 1), data["Price"], test_size=0.15, random_state=123)
X_test

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
len(X_train["Genre"].unique())

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Fit and transform the categorical data
encoded_dataTrain = label_encoder.fit_transform(X_train["BookCategory"])
encoded_dataTest= label_encoder.transform(X_test["BookCategory"])
encoded_dataTrain

## Univariate Variable Analysis



## Basic Data Analysis




## Feature Engineering



In [ ]:
def getPattern(pattern ,text):
    
    match = re.search(pattern, text)
    
    if match:
        return float(match.group())
    else:
        return None  # or any other value or indicator you want for cases where no match is found


In [ ]:
def getPatternString(text):
  # Define a regular expression pattern to match words within brackets
    pattern = r'\(([^)]+)\)'

    # Use re.findall to find all matches in the text
    matches = re.findall(pattern, text)
    for match in matches:
        return match.split()[0]
    

In [ ]:
R_pattern = r'\b\d+\.\d+\b'

getReviews = partial(getPattern, R_pattern)

X_train["Reviews"] = X_train["Reviews"].apply(getReviews)
X_train["Reviews"]

In [ ]:
year_pattern = r'\b\d{4}\b'

getYear = partial(getPattern, year_pattern)

X_train["Year"] = X_train["Edition"].apply(getYear)
X_train["Year"]

In [ ]:
number_pattern = r'\b\d+\b'

getNumber = partial(getPattern, number_pattern)

X_train["Ratings"] = X_train["Ratings"].apply(getNumber)
X_train["Ratings"]

In [ ]:
from sklearn.preprocessing import LabelEncoder

X_train["Type"] = X_train["Genre"].apply(getPatternString)

label_encoder = LabelEncoder()

X_train["Type"] = label_encoder.fit_transform(X_train["Type"])

(X_train["Type"].unique())

In [ ]:
label_encoder2 = LabelEncoder()

X_train["BookCategory"] = label_encoder2.fit_transform(X_train["BookCategory"])

X_train["BookCategory"].unique()

In [ ]:
X_train

In [ ]:
import re

def getType(text):
    words = re.findall(r'\b\w+\b', text)
    return next(word for word in words if words.count(word) == 1)



#X_train["Type2"] = X_train["Edition"].apply(getType)

#X_train["Type2"]

In [ ]:
label_encoder3 = LabelEncoder()

#X_train["Type2"] = label_encoder3.fit_transform(X_train["Type2"])

#X_train

## Feature transformation

In [ ]:
X_train['TAE'] = X_train['Title']+ ' ' + X_train['Author'] + ' ' + X_train['Edition']
X_train['SG'] =  X_train['Genre']+ ' ' + X_train['Synopsis']+ ' ' + X_train['Genre']

X_train["SG"][0]

In [ ]:
from sklearn.decomposition import PCA

tfidf_vectorizerTAE = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizerTAE.fit_transform(X_train['TAE'])

dense_matrix = tfidf_matrix.todense()

TAE = pd.DataFrame(dense_matrix)

num_components = 16  # You can change this based on your requirement
pca = PCA(n_components=num_components)
principal_components = pca.fit_transform(TAE)

pc_columns = [f'TAE-PC_{i+1}' for i in range(num_components)]

pc_df = pd.DataFrame(data=principal_components, columns=pc_columns)

# Concatenate the original DataFrame and the PCA DataFrame

pc_df

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns
import matplotlib.pyplot as plt

# Set the number of clusters for hierarchical clustering
n_clusters = 20  # You can adjust this based on your requirement

# Apply hierarchical clustering on the principal components
hierarchical_clustering = AgglomerativeClustering(n_clusters=n_clusters)
hierarchical_clusters = hierarchical_clustering.fit_predict(pc_df)

pc_df['ClusterTAE'] = hierarchical_clusters

cluster_counts = pc_df['ClusterTAE'].value_counts()

pc_df

In [ ]:
from sklearn.decomposition import PCA

tfidf_vectorizerSG = TfidfVectorizer()

tfidf_matrix = tfidf_vectorizerSG.fit_transform(X_train['SG'])

dense_matrix = tfidf_matrix.todense()

SG = pd.DataFrame(dense_matrix)

num_components = 16  # You can change this based on your requirement
pcaSG = PCA(n_components=num_components)
principal_components = pca.fit_transform(SG)

pc_columns = [f'SG-PC_{i+1}' for i in range(num_components)]

SGDF = pd.DataFrame(data=principal_components, columns=pc_columns)

# Concatenate the original DataFrame and the PCA DataFrame

SGDF

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import seaborn as sns
import matplotlib.pyplot as plt

# Set the number of clusters for hierarchical clustering
n_clusters = 20  # You can adjust this based on your requirement

# Apply hierarchical clustering on the principal components
hierarchical_clusteringSG = AgglomerativeClustering(n_clusters=n_clusters)
hierarchical_clusters = hierarchical_clusteringSG.fit_predict(SGDF)

SGDF['ClusterSG'] = hierarchical_clusters
SGDF

In [ ]:
X_train

In [ ]:
X_train.reset_index(drop=True, inplace=True)
X_train

In [ ]:
lastX_train = pd.concat([pc_df, SGDF], axis=1)
#lastX_train["Type2"] = X_train["Type2"]
lastX_train["Type"] = X_train["Type"]
lastX_train["Year"] = X_train["Year"]
lastX_train["Ratings"] = X_train["Ratings"]
lastX_train["Reviews"] = X_train["Reviews"]
lastX_train["BookCategory"] = X_train["BookCategory"]
lastX_train

In [ ]:
lastX_train.isna().sum()

In [ ]:
lastX_train = lastX_train.dropna()
lastX_train.isna().sum()

In [ ]:
X_test

In [ ]:
tempTest = X_test

getNumber = partial(getPattern, number_pattern)
getReviews = partial(getPattern, R_pattern)
getYear = partial(getPattern, year_pattern)

X_test["Reviews"] = X_test["Reviews"].apply(getReviews)
X_test["Year"] = X_test["Edition"].apply(getYear)
X_test["Ratings"] = X_test["Ratings"].apply(getNumber)
#X_test["Type2"] = X_test["Edition"].apply(getType)
X_test["Type"] = X_test["Genre"].apply(getPatternString)
#X_test["Type2"] = label_encoder3.transform(X_test["Type2"])
X_test["BookCategory"] = label_encoder2.transform(X_test["BookCategory"])
X_test["Type"] = label_encoder.transform(X_test["Type"])
X_test

In [ ]:
X_test['TAE'] = X_test['Title']+ ' ' + X_test['Author'] + ' ' + X_test['Edition']
X_test['SG'] =  X_test['Genre']+ ' ' + X_test['Synopsis']+ ' ' + X_test['Genre']

X_test

In [ ]:
tfidf_matrix = tfidf_vectorizerTAE.fit_transform(X_test['TAE'])

dense_matrix = tfidf_matrix.todense()

TAE = pd.DataFrame(dense_matrix)

num_components = 16  # You can change this based on your requirement
pca = PCA(n_components=num_components)
principal_components = pca.fit_transform(TAE)

pc_columns = [f'TAE-PC_{i+1}' for i in range(num_components)]

pc_df = pd.DataFrame(data=principal_components, columns=pc_columns)

# Concatenate the original DataFrame and the PCA DataFrame

pc_df

In [ ]:
hierarchical_clusters = hierarchical_clustering.fit_predict(pc_df)

pc_df['ClusterTAE'] = hierarchical_clusters

cluster_counts = pc_df['ClusterTAE'].value_counts()

pc_df

In [ ]:
tfidf_matrix = tfidf_vectorizerSG.fit_transform(X_train['SG'])

dense_matrix = tfidf_matrix.todense()

SG = pd.DataFrame(dense_matrix)

num_components = 16  # You can change this based on your requirement
pcaSG = PCA(n_components=num_components)
principal_components = pca.fit_transform(SG)

pc_columns = [f'SG-PC_{i+1}' for i in range(num_components)]

SGDF = pd.DataFrame(data=principal_components, columns=pc_columns)

# Concatenate the original DataFrame and the PCA DataFrame

SGDF

## Modeling

Do not change this part of the code only run it!

In [ ]:
def train(X_train, y_train, X_test, y_test):
    """
    Trains a RandomForestRegressor model and evaluates its performance using the mean squared error (MSE).

    Parameters:
    X (numpy.ndarray or pandas.DataFrame): The training data with (n_rows, n_features) shape.
    y (numpy.ndarray or pandas.Series): The target variable (n_rows, 1) shape.

    Returns:
    float: The mean squared error (MSE) of the predictions(train data) made by the RandomForestRegressor.
    float: The mean squared error (MSE) of the predictions(test data) made by the RandomForestRegressor.
    """
    random_forest_regressor = RandomForestRegressor(criterion='squared_error')
    random_forest_regressor.fit(X_train, y_train)
    
    y_train_pred = random_forest_regressor.predict(X_train)
    y_test_pred = random_forest_regressor.predict(X_test)
    
    mse_train = random_forest_regressor.score(X_train, y_train)
    mse_test = mean_squared_error(y_test, y_test_pred)
    
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)

    #y_pred = random_forest_regressor.predict(X_test)

    return mse_train, mse_test, r2_train ,r2_test

In [ ]:
mse_train, mse_test, r2_train ,r2_test = train(X_train=encoded_dataTrain.reshape(-1, 1) , y_train=y_train , X_test=encoded_dataTest.reshape(-1, 1) , y_test=y_test)
print("Train mse is: {} // Test mse is: {} // Train R2 is: {} // Test R2 is: {}".format(mse_train,mse_test,r2_train ,r2_test))

In [ ]:
random_forest_regressor = RandomForestRegressor(criterion='squared_error')
random_forest_regressor.fit(encoded_data.reshape(-1, 1) ,y_train)
mse_train = random_forest_regressor.score(encoded_data.reshape(-1, 1) ,y_train)
    

## Post processing

Error Analysis: Analyzing the model's errors on the test data can provide valuable insights into its strengths and weaknesses. This involves identifying common error patterns, understanding the causes of errors, and exploring strategies to mitigate them.

Feature Importance Analysis: Identifying the most important features contributing to the model's predictions can help in feature selection and dimensionality reduction. This can lead to a more efficient and interpretable preprocessing pipeline.